In [1]:
import findspark
findspark.init()

from multiprocessing import Pool, cpu_count
from pathlib import Path
import time

import dask.array as da
import numpy as np
import pyqtgraph as pg
import pyspark

import janelia_core.fileio.ahrens_lab as ahrens_lab
from janelia_core.fileio.shared_lab import find_images
from janelia_core.fileio.exp_reader import read_img_file

In [2]:
ephys_folder = Path(r'E:\\bishop_work\\ahrens_two_color\\20161109_2_1_6dpf_GFAP_GC_Huc_RG_GA_CL_fb_OL_f0_0GAIN_20161109_211950\\ephys')
image_folder = Path(r'E:\\bishop_work\\ahrens_two_color\\20161109_2_1_6dpf_GFAP_GC_Huc_RG_GA_CL_fb_OL_f0_0GAIN_20161109_211950\raw')

In [3]:
img_paths = find_images(image_folder, '.h5')

Searching for image files...
Found 12500 images.


In [4]:
n_images = 3000 # Only set to large values if using spark 

In [ ]:
t0 = time.time()
with Pool(min(cpu_count(), n_images)) as pool:
    data = np.array(pool.map(np.max(read_img_file, 0), img_paths[0:n_images]))
t1 = time.time(); 
print("Elapsed time: " + str(t1 - t0))

In [ ]:
exp = ahrens_lab.read_exp(image_folder, ephys_folder)
t0 = time.time()
data2 = da.max(exp.ts_data['imgs']['vls'][0:n_images, :, :, :], 1).compute()
t1 = time.time()
print("Elapsed time: " + str(t1 - t0))

In [13]:
def max_proj(f):
    return np.max(read_img_file(f), 0)

conf = pyspark.SparkConf().setAll([('spark.executor.memory', '5g'), ('spark.executor.cores', '60'),
                                   ('spark.cores.max', '60'), ('spark.driver.memory','200g'), 
                                  ('spark.driver.maxResultSize', '200g')])
sc = pyspark.SparkContext(conf=conf)

In [ ]:
t0 = time.time()
data3 = np.array(sc.parallelize(img_paths[0:n_images]).map(max_proj).collect())
t1 = time.time()
print("Elapsed time: " + str(t1 - t0))

In [ ]:
t0 = time.time()
#data4 = np.array([np.max(read_img_file(f),0) for f in img_paths[0:n_images]])
t1 = time.time()
print("Elapsed time: " + str(t1 - t0))

In [ ]:
#sc.getConf().getAll()
%gui qt
pg.image(data3)

In [ ]:
data3.shape

In [12]:
n_images = 7000

In [11]:
sc.stop()